# Tests for non mathematical attacks

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision
import numpy as np
import random

from PIL import Image

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)

In [ ]:
# Utils import

import sys
import os
rootFolder = os.path.abspath(os.path.dirname(
    os.path.dirname(os.path.abspath(__file__))))
sys.path.append(rootFolder)

from utils.tasks import *
from utils.nonMathAttacks import NonMathAttacks

In [ ]:
nonMathAttacks = NonMathAttacks()

In [ ]:
SEED = 151836

def setSeed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    nonMathAttacks.seed(seed)

setSeed(SEED)

In [ ]:
DATASET_PATH = "../datasets/bing/test"

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

toTensor = transforms.Compose([transforms.ToTensor()])
testDataset = ImageFolder(DATASET_PATH, transform=toTensor)

setSeed(SEED)
testDataLoader = DataLoader(testDataset, batch_size=3, shuffle=True, num_workers=0, pin_memory=True)

## Image tests

In [ ]:
attacks = {
    "Gaussian Noise": nonMathAttacks.gaussianNoise,
    "Box Blur": nonMathAttacks.boxBlur,
    "Invert Color": nonMathAttacks.invertColor,
    "Greyscale": nonMathAttacks.greyscale,
    "Split & Merge RGB": nonMathAttacks.splitMergeRGB,
    "Salt & Pepper": nonMathAttacks.saltAndPepper,
    "Random Black Box": nonMathAttacks.randomBlackBox,
}

In [ ]:
import math

setSeed(SEED)

for (images, labels), attack in zip(testDataLoader, attacks):
    numImages = len(images)

    imageFilter = attacks[attack]

    rows = int(math.sqrt(numImages))
    cols = math.ceil(numImages / rows)

    fig, axs = plt.subplots(
        rows * 2, cols, figsize=(4 * cols, 8 * rows + 1))

    fig.suptitle(attack, fontsize=22)
    
    for row in range(rows):
        for col in range(cols):
            ax = axs[row][col]
            axAttack = axs[rows + row][col]

            img = images[col + row]
            label = labels[col + row]

            img = transforms.ToPILImage()(img).convert("RGB")
            labelName = testDataset.classes[label.numpy()]

            ax.title.set_text(labelName + " (original)")
            ax.imshow(img)

            imgAttack = imageFilter(img)
            axAttack.title.set_text(labelName)
            axAttack.imshow(imgAttack)

    fig.tight_layout()
    fig.subplots_adjust(top=0.90, hspace=0.3)

    plt.show()

## Image for the paper

In [ ]:
img = Image.open("../datasets/bing/test/cat/cat_4049.jpg")
imgOriginal = img.copy()

ticksSize = 18
padTitle = 23
titleSize = 23

plt.rc('xtick', labelsize=ticksSize)
plt.rc('ytick', labelsize=ticksSize)

cols = 4
rows = math.ceil((1 + len(attacks)) / cols)

fig, axs = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows + 1))

ax = axs[0][0]
ax.set_title("Original", pad=padTitle, fontsize=titleSize)
ax.imshow(imgOriginal)

for attack, row in zip(attacks, range(len(attacks))):
    row = row + 1
    ax = axs[row // cols][row % cols]

    imageFilter = attacks[attack]
    imgAttack = imageFilter(img)

    ax.set_title(attack, pad=padTitle, fontsize=titleSize)
    ax.imshow(imgAttack)

fig.tight_layout()

plt.show()

for fmt in ["pdf", "png"]:
    fig.savefig("plots/{fmt}/catNonMathAttacks.{fmt}".format(fmt=fmt),
                facecolor=None,
                dpi=500,
                bbox_inches="tight",
                pad_inches=.3)
